# Clustering point cloud data from the Imaging Radar

This notebook is intended to cluster the point cloud data (PCD) from the imaging radar using the K-Means clustering algorithm. The PCD is recieved from the data acquisition team in the .csv format.


### Importing all the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib qt
from sklearn.cluster import KMeans

### Importing the PCD csv file

In [ ]:
data = pd.read_csv("PCDSample3_mergedframes.csv")
data.head()

x = data[['x','y','z']].values
H = 0.5 #Height of the sensor from the ground
x[:,2] = x[:,2]*-1
# #print(x[:,2])
# #print(len(x[:,2]))
# #print(x.shape)
# print(x)
#x

In [ ]:
#data_filtered = data[data['z']>0.5]
#data_filtered = data['z']
#data_filtered
#data_filtered.shape

#Because the 'z' takes data from the previous version before multiplying it with -1

### Data cleaning

The data points below the level of sensor are eliminated and a new array of co-ordinates are stored in array 'kk'

In [ ]:
k = []

for i in range(len(x[:,2])):
    if x[i,2]>H:
        k.append(x[i])
        #y = np.asarray(k, dtype=np.float32)
kk=np.array(k)        
#print(x[:,2])
#print(len(x[:,2]))
print(kk)
print(len(kk))
print(kk.shape)

In [ ]:
#y=[]
#for i in range(len(x[:,2])):
    #if x[i,2]<0:
       # y[i]=np.delete(x,i,0)
#print(y)
#print(len(y))

### Estimation of optimal 'K' value using the Elbow rule

In [ ]:
WCSS = []

for i  in range(1,11):
    kmeans_model = KMeans(n_clusters = i, init = 'k-means++')
    kmeans_model.fit(kk)
    WCSS.append(kmeans_model.inertia_)
    
    
fig = plt.figure(figsize = (7,7))
plt.plot(range(1,11),WCSS, linewidth=4, markersize=12,marker='o',color = 'red')
plt.xticks(np.arange(11))
plt.xlabel("Number of Clusters")
plt.ylabel("WCSS")
plt.show()

## K-Means clustering

The number of clusters is initialized and the algorithm is defined.

In [ ]:
kmeans_model = KMeans(n_clusters=3, init = "k-means++", max_iter = 300, n_init = 10, random_state = 0).fit(kk)
y_clusters = kmeans_model.fit_predict(kk)


#check number of clusters and number of points in each cluster
sns.countplot(y_clusters)
print(kk[y_clusters == 0,0][1])
print(kk[y_clusters == 0,1][1])
print(kk[y_clusters == 0,2][1])

### Visualize the clusters

In [ ]:
fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(kk[y_clusters == 0,0],kk[y_clusters == 0,1],kk[y_clusters == 0,2], s = 40 , color = 'blue', label = "cluster 0")
ax.scatter(kk[y_clusters == 1,0],kk[y_clusters == 1,1],kk[y_clusters == 1,2], s = 40 , color = 'orange', label = "cluster 1")
ax.scatter(kk[y_clusters == 2,0],kk[y_clusters == 2,1],kk[y_clusters == 2,2], s = 40 , color = 'green', label = "cluster 2")
#ax.scatter(x[y_clusters == 3,0],x[y_clusters == 3,1],x[y_clusters == 3,2], s = 40 , color = 'brown', label = "cluster 3")
#ax.scatter(x[y_clusters == 4,0],x[y_clusters == 4,1],x[y_clusters == 4,2], s = 40 , color = 'black', label = "cluster 4")
#ax.scatter(x[y_clusters == 5,0],x[y_clusters == 5,1],x[y_clusters == 5,2], s = 40 , color = 'red', label = "cluster 5")
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.legend()
plt.show()